In [239]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [240]:
load_dotenv(override=True)

True

In [241]:
api_key = os.getenv("OPENAI_API_KEY")
files = [os.getenv("SOURCE_PDF")]
pages = []
store = os.getenv("SOURCE_STORE")

# Files

In [242]:
for file in files:
    loader = PyPDFLoader(file)
    pages.extend(loader.load())

# Recursive TextSplit

In [243]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [244]:
documents = recur_split.split_documents(pages)

# Embbiding

In [245]:
embeddings_model = OpenAIEmbeddings()

# Vector Store

In [246]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=store
)

# LLM

In [247]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [248]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [249]:
question = "O que é Identificar oportunidades?"

# Answer

In [250]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?',
 'result': 'Identificar oportunidades é usar a sua imaginação e as suas habilidades para visualizar oportunidades de criação de valor, como a identificação de necessidades e desafios que precisam de ser atendidos.'}

# Prompt

In [251]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
- A disciplina;
- O conteúdo;
- O objetivo da aula.

Com base no documento ENTRECOMP (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Retorne apenas os **nomes das competências**, separados por vírgula e a última separada por e.
Exemplo de formato: Visão, Pensamento ético e sustentável e Valorizar ideias.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

# Chain

In [252]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [253]:
question = "Aula de <Educação Religiosa>, sendo o conteúdo <Nomes> e o objetivo da aula é <Importância dos Nomes>."

# Answer

In [254]:
answer = chat_chain.invoke({"query": question})

In [255]:
print(answer["result"])

Visão, Valorizar ideias e Criatividade.


In [256]:
answer["source_documents"]

[Document(metadata={'author': 'Roselaine Monteiro Moraes', 'producer': 'Microsoft® Word para Microsoft 365', 'page_label': '1', 'moddate': '2025-04-28T16:59:30-03:00', 'source': 'D:\\Visual Code\\RAG\\pdf\\entrecomp.pdf', 'page': 0, 'doc_id': 1, 'creationdate': '2025-04-28T16:59:30-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'total_pages': 2}, page_content='significativos. \n \nVisão: é trabalhar para uma visão de futuro, imaginar o futuro, desenvolver um olhar \npara transformar ideias em ações e visualizar cenários futuros para ajudar a orientar \nesforços e iniciativas. \n \nValorizar ideias: é fazer o máximo com as ideias e as oportunidades, avaliar que \nvalor existe em termos sociais, culturais e econômicos, reconhecer o potencial de \ncriação de valor de uma ideia e identificar formas adequadas de tirar o máximo dela.'),
 Document(metadata={'moddate': '2025-04-28T16:59:30-03:00', 'creationdate': '2025-04-28T16:59:30-03:00', 'source': 'D:\\Visual Code\\RAG\\pdf\\entr

# Debug

In [257]:
set_debug(True)

question = "Aula de <Matemática>, sendo o conteúdo <Frações> e o objetivo da aula é <Utilização em cálculo de juros>."
answer = chat_chain.invoke({"query": question})
answer["source_documents"]

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Aula de <Matemática>, sendo o conteúdo <Frações> e o objetivo da aula é <Utilização em cálculo de juros>."
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Aula de <Matemática>, sendo o conteúdo <Frações> e o objetivo da aula é <Utilização em cálculo de juros>.",
  "context": "Educação financeira e econômica: é desenvolver conhecimentos financeiros \ne econômicos, saber calcular o custo de transformar uma ideia numa atividade criadora \nde valor. Planejar, colocar em prática e avaliar decisões financeiras ao longo do tempo. \nGerir as finanças para assegurar que a atividade de criação de valor possa manter-se a \nlongo prazo. \n \nMobilização de pessoas: é inspirar, entusiasmar e mobilizar parceiros\n\nEducação financeira e econô